In [ ]:
##### import gym
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv

VtoM_trans = 2048  #Mbps以5G傳輸速度為範本
VtoV_trans = 3072  #Mbps
VtoC_trans = 2048  #Mbps

car_process = 300  #Mbps
MEC_process = 3200 #Mbps
break_count = 0
if __name__ == '__main__':
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.99, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    n_games = 1500

    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):

        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])

        if score > best_score : #如果平均分數大於最佳分數 
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('best_score_latency.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
        if avg_score > -0.620:
            break_count += break_count
            print("break_count = ",break_count)
        if break_count == 5 :
            break
    filename = 'offloading_ddqn1.png'

    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(1500):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('avg_score_0330_0447.csv',encoding='utf-8', index = False)


In [ ]:
#local computing 
import numpy as np
import random
import pandas as pd
import csv

time_slot = 500
epoch = 1500
car_process = 753  # Mbps
csv_latency = np.zeros([epoch], dtype="float32")

latency = np.zeros([time_slot], dtype="float32")  # Move this line outside the loop

for i in range(epoch):
    mu_cpu, sigma_cpu = car_process, 10
    car_cpu = np.random.normal(mu_cpu, sigma_cpu, time_slot)
    car_cpu = np.clip(car_cpu, car_process - 50, car_process + 50)
    car_cpu = car_cpu.astype(int)

    mu, sigma = 750, 100
    car1_data = np.random.normal(mu, sigma, time_slot)
    car1_data = np.clip(car1_data, 500, 1000)
    car1_data = car1_data.astype(int)

    before_car1_data = 0

    latency_total = 0

    for step_number in range(time_slot):  # Change the variable name from setp_number to step_number
        waiting_data_number = before_car1_data
        latency_process = car1_data[step_number] / car_cpu[step_number]
        latency_queue = waiting_data_number / car_cpu[step_number]

        latency[step_number] = latency_process + latency_queue
        before_car1_data = (latency[step_number] - 1) * car_cpu[step_number]

        if before_car1_data < 0:
            before_car1_data = 0

    for j in range(time_slot):  # Use a different variable name here
        latency_total += latency[j]
    avg_latency = latency_total / time_slot
    #print(avg_latency)
    
    if avg_latency < 4.5 and avg_latency > 4:
        print(avg_latency)
        latency_pd = pd.DataFrame(data=latency)
        latency_pd.to_csv('503_latency.csv', encoding='utf-8', index=False)
        #print(latency)


    csv_latency[i] = avg_latency

print(csv_latency)

csv_latency = pd.DataFrame(data=csv_latency)
csv_latency.to_csv('local_latency.csv', encoding='utf-8', index=False)

In [ ]:
#cloud computing
import numpy as np
import random
import pandas as pd
import csv

time_slot = 500
epoch = 1500

csv_latency = np.zeros([epoch], dtype="float32")

latency = np.zeros([time_slot], dtype="float32")  # Move this line outside the loop

for i in range(epoch):
    mu_cpu, sigma_cpu = 1024, 100
    car_cpu = np.random.normal(mu_cpu, sigma_cpu, time_slot)
    car_cpu = np.clip(car_cpu, car_cpu - 50, car_cpu + 50)
    car_cpu = car_cpu.astype(int)

    mu, sigma = 750, 100
    car1_data = np.random.normal(mu, sigma, time_slot)
    car1_data = np.clip(car1_data, 500, 1000)
    car1_data = car1_data.astype(int)

    before_car1_data = 0

    latency_total = 0

    for step_number in range(time_slot):  # Change the variable name from setp_number to step_number

        latency_process = car1_data[step_number] / car_cpu[step_number]
        latency[step_number] = latency_process 


    for j in range(time_slot):  # Use a different variable name here
        latency_total += latency[j]
    avg_latency = latency_total / time_slot
    #print(avg_latency)

    latency_pd = pd.DataFrame(data=latency)
    latency_pd.to_csv('cloud_latency.csv', encoding='utf-8', index=False)
        #print(latency)


    csv_latency[i] = avg_latency

print(csv_latency)

csv_latency = pd.DataFrame(data=csv_latency)
csv_latency.to_csv('total_cloud_latency.csv', encoding='utf-8', index=False)

In [ ]:
#####!@#$%錯誤的kstest 算法
import numpy as np
from scipy.stats import kstest, norm
import pandas as pd

# 讀取csv檔案
data = pd.read_csv('best_score_latency_0409.csv')

# 將數據轉換成numpy陣列
data_array = np.array(data['0'])

# 執行KS檢驗
statistic, p_value = kstest(data_array, 'norm')

# 計算KS檢驗結果的門檻值
alpha = 0.01
n = len(data_array)
ks_threshold = np.sqrt(-0.5 * np.log(alpha / 2)) / np.sqrt(n)

# 判斷KS檢驗結果是否通過
if statistic < ks_threshold:
    print("P-value: ", p_value)
    print(f"KS test statistic D = {statistic}")
    print("The data follows a normal distribution")
else:
    print("P-value: ", p_value)
    print(f"KS test statistic D = {statistic}")
    print("The data does not follow a normal distribution")


In [ ]:
#計算一般常態分佈的 kstest 資料路徑 /plots/perason_kstest_plot/normal_distribution
import numpy as np
from scipy.stats import kstest, poisson, norm
import pandas as pd

# 讀取 csv 檔案
data = pd.read_csv('plots/perason_kstest_plot/normal_distribution.csv', header=None)
#data = pd.read_csv('RE_plot/DDQN_best_latency.csv', header=None)

# 轉換為 numpy array
sample = np.array(data[0])

# 計算平均值
sample_mean = np.mean(sample)
loc, scale = norm.fit(sample)
n = norm(loc=loc, scale=scale)

# 計算 KS test
#D, p_value = kstest(sample, norm(sample_mean).cdf)
D, p_value = kstest(sample, n.cdf)

# 列印結果
print('KS test statistic D:', D)
print('p-value:', p_value)


In [ ]:
#計算DDQN kstest 資料路徑 /plots/perason_kstest_plot/poisson_plot_latency
import numpy as np
from scipy.stats import kstest, poisson, norm
import pandas as pd

# 讀取 csv 檔案
#data = pd.read_csv('0.csv', header=None)
data = pd.read_csv('plots/perason_kstest_plot/poisson_plot_latency.csv', header=None)

# 轉換為 numpy array
sample = np.array(data[0])

# 計算平均值
sample_mean = np.mean(sample)
loc, scale = norm.fit(sample)
n = norm(loc=loc, scale=scale)

# 計算 KS test
#D, p_value = kstest(sample, norm(sample_mean).cdf)
D, p_value = kstest(sample, n.cdf)

# 列印結果
print('KS test statistic D:', D)
print('p-value:', p_value)


In [ ]:
#python 版本的Cullen and Frey graph圖*********
from scipy.stats import probplot
import pandas as pd
import matplotlib.pyplot as plt

# 讀取 csv 檔案
df = pd.read_csv('plots/perason_kstest_plot/poisson_plot_latency0.csv')

# 使用 probplot 函數繪製 Cullen and Frey graph
fig, ax = plt.subplots(figsize=(8, 6))
probplot(df['0'], plot=ax, dist='norm', fit=True)
ax.set_title('Pearson Plot for Normal Distribution')
filename = 'python_Cullen_and_Frey_Graph.png'
figure_file = 'plots/perason_kstest_plot/' + filename
plt.savefig(figure_file, dpi=1200,bbox_inches='tight')
plt.show()


In [ ]:
# 生成常態分佈數據比對DDQN的數據
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import kurtosis, skew
import pandas as pd

data = np.random.normal(0.6, 0.08, size=1492)

best_latency_csv=pd.DataFrame(data=data)
best_latency_csv.to_csv('plots/perason_kstest_plot/normal_distribution.csv',encoding='utf-8', index = False)


In [ ]:
#DDQN值方圖
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# 生成一組隨機數據
data = pd.read_csv('plots/perason_kstest_plot/poisson_plot_latency0.csv')#在KStest資料夾
data = np.array(data['0'])
# 進行正態分布擬合
mu, std = norm.fit(data)
fig, ax = plt.subplots(figsize=(8, 6))
ax.set_title('DDQN Value Square Chart')
# 繪製樣本數據的直方圖
plt.hist(data, bins=30, density=True, alpha=0.6, color='g')

# 繪製正態分布的概率密度函數
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

filename = 'Value_Square_Chart.png'
figure_file = 'plots/perason_kstest_plot/' + filename
plt.savefig(figure_file, dpi=1200,bbox_inches='tight')
plt.show()

# 輸出擬合後的均值和標準差
print("Fitted mean = {:.2f}".format(mu))
print("Fitted standard deviation = {:.2f}".format(std))

In [ ]:
#常態分布值方圖
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# 生成一組隨機數據
data = np.random.randn(1492)

# 進行正態分布擬合
mu, std = norm.fit(data)

# 繪製樣本數據的直方圖
plt.hist(data, bins=30, density=True, alpha=0.6, color='g')

# 繪製正態分布的概率密度函數
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)

plt.show()

# 輸出擬合後的均值和標準差
print("Fitted mean = {:.2f}".format(mu))
print("Fitted standard deviation = {:.2f}".format(std))

In [2]:
##### 802.11P傳輸DDQN
import numpy as np
from dueling_ddqn_tf2 import Agent
from dueling_ddqn_tf2 import DuelingDeepQNetwork
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv

VtoM_trans = 20#110  #Mbps以5G傳輸速度為範本
VtoV_trans = 20#165  #Mbps
VtoC_trans = 20#110  #Mbps

car_process = 40  #Mbps
MEC_process = 170 #Mbps

if __name__ == '__main__':
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    tf.compat.v1.enable_eager_execution()
    #tf.enable_eager_execution()
    #env = gym.make('LunarLander-v2')
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.99, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    
    n_games = 1500
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -3#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('80211P_trans_latency.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
        if avg_score > -0.62:
            break_count = break_count + 1
            print("break_count = ",break_count)
        if break_count > 200 and avg_score < -0.62 and i>1000:
            n_games = i+1
            print("n_games = ",n_games)
            break
    filename = 'offloading_ddqn_0409.png'

    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('80211P_trans_avg_score.csv',encoding='utf-8', index = False)


break_count =  0
action_number =  192 103 93 112
best_score loading =  -2.990204494348663
episode:  0 score: -2.99020  average score -2.99020 epsilon 0.56


KeyboardInterrupt: 

In [ ]:
##### 5G 傳輸
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv
import time
import pydot
from tensorflow.keras.utils import plot_model

NR_date_rate = 8*8*(948/1024)*(1*12)*(0.001*(14*2**1))*(1-0.14)

VtoM_trans = NR_date_rate*7  #110Mbps以5G傳輸速度為範本
VtoV_trans = NR_date_rate*10  #165Mbps
VtoC_trans = NR_date_rate*7  #110Mbps

car_process = 40  #Mbps
MEC_process = 165 #Mbps


if __name__ == '__main__':
    start = time.time()
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    tf.compat.v1.enable_eager_execution()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.9, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)

    
    n_games = 2000
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('plots/other_data/best_score_latency_0.8.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
        if i>1000: 
            if avg_score > -0.62:
                break_count = break_count + 1
                print("break_count = ",break_count)
        if break_count > 100 and avg_score > -0.62 :
            n_games = i+1
            print("n_games = ",n_games)
            break
    filename = 'offloading_ddqn_0.8.png'
    end = time.time()
    print("總共所花的時間",format(end-start))
    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('plots/learning_rate/DDQN_avg_score_0.8.csv',encoding='utf-8', index = False)


In [ ]:
##learning rate 0.0001 

In [ ]:
##### 5G 傳輸
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv
import time
VtoM_trans = 110  #Mbps以5G傳輸速度為範本
VtoV_trans = 165  #Mbps
VtoC_trans = 110  #Mbps

car_process = 60  #Mbps
MEC_process = 170 #Mbps


if __name__ == '__main__':
    start = time.time()
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    tf.compat.v1.enable_eager_execution()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.90, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    n_games = 1500
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('plots/perason_kstest_plot/best_score_latency_60mbps.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
#         if i>1000: 
#             if avg_score > -0.62:
#                 break_count = break_count + 1
#                 print("break_count = ",break_count)
#         if break_count > 100 and avg_score > -0.62 :
#             n_games = i+1
#             print("n_games = ",n_games)
#             break
    filename = 'offloading_ddqn_60mbps.png'
    end = time.time()
    print("總共所花的時間",format(end-start))
    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('plots/all_avg_score_offloading_plot/DDQN_avg_score_60mbps.csv',encoding='utf-8', index = False)

In [ ]:
##### 5G 傳輸
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv
import time
VtoM_trans = 110  #Mbps以5G傳輸速度為範本
VtoV_trans = 165  #Mbps
VtoC_trans = 110  #Mbps

car_process = 80  #Mbps
MEC_process = 170 #Mbps


if __name__ == '__main__':
    start = time.time()
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    tf.compat.v1.enable_eager_execution()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.9, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    n_games = 1500
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('plots/perason_kstest_plot/best_score_latency_80mbps.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
#         if i>1000: 
#             if avg_score > -0.62:
#                 break_count = break_count + 1
#                 print("break_count = ",break_count)
#         if break_count > 100 and avg_score > -0.62 :
#             n_games = i+1
#             print("n_games = ",n_games)
#             break
    filename = 'offloading_ddqn_0.8.png'
    end = time.time()
    print("總共所花的時間",format(end-start))
    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('plots/all_avg_score_offloading_plot/best_score_latency_80mbps.csv',encoding='utf-8', index = False)

In [ ]:
##### 5G 傳輸
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv
import time
VtoM_trans = 110  #Mbps以5G傳輸速度為範本
VtoV_trans = 165  #Mbps
VtoC_trans = 110  #Mbps

car_process = 120  #Mbps
MEC_process = 170 #Mbps


if __name__ == '__main__':
    start = time.time()
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    tf.compat.v1.enable_eager_execution()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.9, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    n_games = 1500
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('plots/perason_kstest_plot/best_score_latency_120mbps.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
#         if i>1000: 
#             if avg_score > -0.62:
#                 break_count = break_count + 1
#                 print("break_count = ",break_count)
#         if break_count > 100 and avg_score > -0.62 :
#             n_games = i+1
#             print("n_games = ",n_games)
#             break
    filename = 'offloading_ddqn_0.85.png'
    end = time.time()
    print("總共所花的時間",format(end-start))
    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('plots/all_avg_score_offloading_plot/best_score_latency_120mbps.csv',encoding='utf-8', index = False)

In [ ]:
##### 5G 傳輸
import numpy as np
from dueling_ddqn_tf2 import Agent
from utils import plotLearning
from enviroment_offloading import offloading_env
import tensorflow as tf
import pandas as pd
import csv
import time
VtoM_trans = 110  #Mbps以5G傳輸速度為範本
VtoV_trans = 165  #Mbps
VtoC_trans = 110  #Mbps

car_process = 100  #Mbps
MEC_process = 170 #Mbps


if __name__ == '__main__':
    start = time.time()
    #tf.compat.v1.disable_eager_execution()#關閉tensorflow2裡的eager_execution，eager_execution這會讓tf執行的很慢
    #env = gym.make('LunarLander-v2')
    tf.compat.v1.enable_eager_execution()
    env = offloading_env(VtoM_trans,VtoV_trans,VtoC_trans,car_process,MEC_process)
    agent = Agent(lr=0.0001, gamma=0.9, n_actions = env.action_space, epsilon=1.0,
                  batch_size=64, input_dims=env.observation_space)
    n_games = 1500
    break_count = 0
    ddqn_scores = []
    eps_history = []
    best_score = -1.6#平均要超過才能保存
    print("break_count = ",break_count)
    for i in range(n_games):
        
        action0_number = 0
        action1_number = 0
        action2_number = 0
        action3_number = 0
        done = False
        observation = env.reset()
        setp_number = -1
        car1data = 0
        car2data = 0
        MECdata = 0
        score = 0
        done = False
        score = 0
        observation = env.reset()
        while not done:
            setp_number =setp_number +1
            action = agent.choose_action(observation)
            observation_, reward, done, info,car1data ,car2data ,MECdata  ,csv_latency = env.step(action,setp_number,car1data ,car2data ,MECdata)            
#             print("第",setp_number,"個action = ", action)
#             print("延遲 = ",reward)
#             print("第一台車現在的數據輛 = ",observation_[0])
#             print("第一台車之前的數據輛 = ",car1data)  
            
#             print("第二台車現在的數據輛 = ",observation_[1])
#             print("第二台車之前的數據輛 = ",car2data)
            
#             print("MEC現在的數據輛 = ",observation_[2])   
#             print("MEC之前的數據輛 = ",MECdata)
#             print("========================================")
            score += reward #這次得到的分數就是這一圈epoch的所有reward加總的值,然後迴圈一直更新,直到done為true,就是跑完這次epoch的分數
            #score = score - reward
#             print("score = ",score)
            agent.store_transition(observation, action, reward, observation_, done) #把這個迴圈的資料(觀察,動作,獎勵,下一個觀察,終端狀態)存進replay buffer
            observation = observation_ #要在while裡更新新的狀態,不然會一直是舊得observation進行learn
            
            agent.learn() #迭代中進行TD學習
            if action  == 0 :
                action0_number += 1
            elif action  == 1:
                action1_number += 1
            elif action  == 2:
                action2_number += 1
            elif action  == 3:
                action3_number += 1
        print("action_number = ", action0_number,action1_number,action2_number,action3_number)
        
        
        eps_history.append(agent.epsilon)
        score = score/(env.data_time_slot)
        ddqn_scores.append(score)
        avg_score = np.mean(ddqn_scores[-50:])
        

                
        
        
        if score > best_score : #如果平均分數大於最佳分數
            best_score = score #覆蓋掉最佳分數
            best_latency_csv=pd.DataFrame(data=csv_latency)
            print("best_score loading = ",best_score)
            best_latency_csv.to_csv('plots/perason_kstest_plot/best_score_latency_100mbps.csv',encoding='utf-8', index = False)
            
        print('episode: ', i,'score: %.5f' % score,' average score %.5f' % avg_score , 'epsilon %.2f' % agent.epsilon )
#         if i>1000: 
#             if avg_score > -0.62:
#                 break_count = break_count + 1
#                 print("break_count = ",break_count)
#         if break_count > 100 and avg_score > -0.62 :
#             n_games = i+1
#             print("n_games = ",n_games)
#             break
    filename = 'offloading_ddqn_0.8.png'
    end = time.time()
    print("總共所花的時間",format(end-start))
    x = [i+1 for i in range(n_games)]
    plotLearning(x, ddqn_scores, eps_history, filename)
    for i in range(n_games):
        ddqn_scores[i] =-ddqn_scores[i] 
        
    N = len(ddqn_scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(ddqn_scores[max(0, t-50):(t+1)])
    print(running_avg)    
    test=pd.DataFrame(data=running_avg)
    test.to_csv('plots/all_avg_score_offloading_plot/best_score_latency_100mbps.csv',encoding='utf-8', index = False)